In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install sentencepiece


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 69.9 MB/s 
     |████████████████████████████████| 101 kB 10.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 5.2 MB/s 


In [ ]:
!pip install -q datasets==2.0.0

     |████████████████████████████████| 325 kB 5.0 MB/s 
     |████████████████████████████████| 115 kB 59.6 MB/s 
     |████████████████████████████████| 212 kB 74.5 MB/s 
     |████████████████████████████████| 141 kB 88.1 MB/s 
     |████████████████████████████████| 127 kB 80.9 MB/s 


In [ ]:
!pip install -q seqeval


     |████████████████████████████████| 43 kB 995 kB/s 


In [ ]:
!pip install -q pyyaml==5.1
# workaround: install old version of pytorch since detectron2 hasn't released packages for pytorch 1.9 (issue: https://github.com/facebookresearch/detectron2/issues/3158)
!pip install -q torch==1.11.0+cu113 torchvision==0.9.0+cu113 -f https://download.pytorch.org/whl/torch_stable.html

# install detectron2 that matches pytorch 1.8
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#!pip install -q detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

#!git clone https://github.com/facebookresearch/detectron2.git
#!python -m pip install -e detectron2

     |████████████████████████████████| 274 kB 5.0 MB/s 
     |████████████████▎               | 834.1 MB 1.2 MB/s eta 0:11:37tcmalloc: large alloc 1147494400 bytes == 0x3a6a2000 @  0x7ff4c63ce615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████▋           | 1055.7 MB 85.9 MB/s eta 0:00:07tcmalloc: large alloc 1434370048 bytes == 0x7ecf8000 @  0x7ff4c63ce615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |██████████████████████████▏     | 1336.2 MB 1.2 MB/s eta 0:04:20tcmalloc: large alloc 1792966656 bytes == 0x3b2a000

In [ ]:
#imports
import os
import glob
import json
import random
from pathlib import Path
from difflib import SequenceMatcher


os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8" 
import torch 
import numpy as np
import random
seed = 7
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.use_deterministic_algorithms(True)

import cv2
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
from IPython.display import display
import matplotlib
from PIL import Image
from matplotlib import pyplot, patches
from transformers import  LayoutLMv2Processor, LayoutXLMTokenizer, LayoutXLMProcessor
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D

from datasets import load_dataset,Dataset

from transformers import LayoutLMv2ForTokenClassification, TrainingArguments, Trainer
from datasets import load_metric
import numpy as np

In [ ]:


datasets = load_dataset("darentang/sroie",cache_dir=None)
#datasets = load_dataset("nielsr/funsd",cache_dir=None)
#datasets = load_dataset("nielsr/funsd-layoutlmv3")

##creating a dataloader

from torch.utils.data import DataLoader

# train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=2)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset sroie downloaded and prepared to /root/.cache/huggingface/datasets/darentang___sroie/sroie/1.0.0/26ed9374c9a15a1d2f44fd8886f679076e1a1fd7da2d53726d6e58a99436c506. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
#labels
labels = datasets['train'].features['ner_tags'].feature.names
print(labels)
#labels
id2label = {v: k for v, k in enumerate(labels)}
label2id = {k: v for v, k in enumerate(labels)}
label2id


['O', 'B-COMPANY', 'I-COMPANY', 'B-DATE', 'I-DATE', 'B-ADDRESS', 'I-ADDRESS', 'B-TOTAL', 'I-TOTAL']


{'B-ADDRESS': 5,
 'B-COMPANY': 1,
 'B-DATE': 3,
 'B-TOTAL': 7,
 'I-ADDRESS': 6,
 'I-COMPANY': 2,
 'I-DATE': 4,
 'I-TOTAL': 8,
 'O': 0}

#Text: The original word text.

Lemma: The base form of the word.

POS: The simple UPOS part-of-speech tag.

Tag: The detailed part-of-speech tag.

Dep: Syntactic dependency, i.e. the relation between tokens.

Shape: The word shape – capitalization, punctuation, digits.

is alpha: Is the token an alpha character?

is stop: Is the token part of a stop list, i.e. the most common words of the language?

In [ ]:
datasets["train"]

Dataset({
    features: ['id', 'words', 'bboxes', 'ner_tags', 'image_path'],
    num_rows: 626
})

In [ ]:
## preprocess the dataset

#processor = LayoutLMv2Processor.from_pretrained("microsoft/layoutlmv2-base-uncased", revision="no_ocr")
from transformers import LayoutLMv2FeatureExtractor, LayoutLMv2Tokenizer, LayoutLMv2Processor, AutoTokenizer
feature_extractor = LayoutLMv2FeatureExtractor(apply_ocr=False)
tokenizer = LayoutXLMTokenizer.from_pretrained('microsoft/layoutxlm-base')
#tokenizer = AutoTokenizer.from_pretrained('microsoft/layoutxlm-base')

processor = LayoutXLMProcessor(feature_extractor, tokenizer)

def read_image_from_dataset(img_data_str: str): 
    img_data_str_enc = img_data_str.encode("utf-8") 
    img_data_bytes = base64.b64decode(img_data_str_enc) 
    img = Image.open(io.BytesIO(img_data_bytes)).convert("RGB") 
    return np.array(img)

# we need to define custom features
features = Features({
    'image': Array3D(dtype="int64", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'token_type_ids': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(ClassLabel(names=labels)),
})

def preprocess_data(examples):
  
  words = examples['words']
  boxes = examples['bboxes']
  word_labels = examples['ner_tags']
  images = [Image.open(path).convert("RGB") for path in examples['image_path']]

  encoded_inputs = processor(images, words, boxes=boxes, word_labels=word_labels,
                             padding="max_length", truncation=True, max_length=512, return_token_type_ids=True) 
 
  return encoded_inputs

train_dataset = datasets["train"].map(preprocess_data, batched=True, remove_columns=datasets["train"].column_names,
                                      features=features)
test_dataset = datasets["test"].map(preprocess_data, batched=True, remove_columns=datasets["test"].column_names,
                                      features=features)


train_dataset.set_format(type="torch")
test_dataset.set_format(type="torch")

##creating a dataloader

from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8)


Parameter 'function'=<function preprocess_data at 0x7f306b3b88c0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:

# train_dataset.set_format(type="torch")
# test_dataset.set_format(type="torch")

# ##creating a dataloader

# from torch.utils.data import DataLoader

# train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=2)





model = LayoutLMv2ForTokenClassification.from_pretrained('microsoft/layoutxlm-base',
                                                                      num_labels=len(label2id))

# Set id2label and label2id 
model.config.id2label = id2label
model.config.label2id = label2id

# Metrics
metric = load_metric("seqeval")
return_entity_level_metrics = True



using `AvgPool2d` instead of `AdaptiveAvgPool2d`
Some weights of the model checkpoint at microsoft/layoutxlm-base were not used when initializing LayoutLMv2ForTokenClassification: ['layoutlmv2.visual.backbone.bottom_up.res4.14.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.8.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.20.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res2.0.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res3.1.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res3.3.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.6.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.11.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res5.1.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.11.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

class FunsdTrainer(Trainer):
    def get_train_dataloader(self):
      return train_dataloader

    def get_test_dataloader(self, test_dataset):
      return test_dataloader


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))

368233225


In [ ]:
args = TrainingArguments(
    output_dir="layoutlmv2-finetuned-agi-v2", # name of directory to store the checkpoints
    max_steps=1000, # we train for a maximum of 1,000 batches
    warmup_ratio=0.1, # we warmup a bit
    fp16=False, # we use mixed precision (less memory consumption)
    push_to_hub=False, # after training, we'd like to push our model to the hub
     # this is the name we'll use for our model on the hub
)

# Initialize our Trainer
trainer = FunsdTrainer(
    model=model,
    args=args,
    compute_metrics=compute_metrics,
)



max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 626
  Num Epochs = 13
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1000


Step,Training Loss
500,0.110400
1000,0.004700


Saving model checkpoint to layoutlmv2-finetuned-agi-v2/checkpoint-500
Configuration saved in layoutlmv2-finetuned-agi-v2/checkpoint-500/config.json
Model weights saved in layoutlmv2-finetuned-agi-v2/checkpoint-500/pytorch_model.bin
Saving model checkpoint to layoutlmv2-finetuned-agi-v2/checkpoint-1000
Configuration saved in layoutlmv2-finetuned-agi-v2/checkpoint-1000/config.json
Model weights saved in layoutlmv2-finetuned-agi-v2/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1000, training_loss=0.05755462646484375, metrics={'train_runtime': 899.5398, 'train_samples_per_second': 8.893, 'train_steps_per_second': 1.112, 'total_flos': 4280043103100928.0, 'train_loss': 0.05755462646484375, 'epoch': 12.66})

In [ ]:
predictions, labels, metrics = trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 347
  Batch size = 8


In [ ]:
metrics

{'test_ADDRESS_f1': 0.9283667621776505,
 'test_ADDRESS_number': 347,
 'test_ADDRESS_precision': 0.9230769230769231,
 'test_ADDRESS_recall': 0.9337175792507204,
 'test_COMPANY_f1': 0.9470672389127325,
 'test_COMPANY_number': 347,
 'test_COMPANY_precision': 0.9403409090909091,
 'test_COMPANY_recall': 0.9538904899135446,
 'test_DATE_f1': 0.9870503597122302,
 'test_DATE_number': 347,
 'test_DATE_precision': 0.985632183908046,
 'test_DATE_recall': 0.9884726224783862,
 'test_TOTAL_f1': 0.9274537695590328,
 'test_TOTAL_number': 347,
 'test_TOTAL_precision': 0.9157303370786517,
 'test_TOTAL_recall': 0.9394812680115274,
 'test_loss': 0.029717469587922096,
 'test_overall_accuracy': 0.9961396649427137,
 'test_overall_f1': 0.9474060822898032,
 'test_overall_precision': 0.9410092395167022,
 'test_overall_recall': 0.9538904899135446,
 'test_runtime': 11.3549,
 'test_samples_per_second': 30.559,
 'test_steps_per_second': 3.875}

In [ ]:
## layoutlm 11 M base, 2 epoch